In [1]:
import pytorch_lightning as pl

In [2]:
from src.dataset import ImageDataModule, PyradiomicsDataModule
from src.baseline import BaselineClf


In [3]:
train_loader = ImageDataModule()
trainer = pl.Trainer(accelerator="gpu", devices=1)
model = BaselineClf()



C:\Users\Julia\miniconda3\envs\ml4h\lib\site-packages\pytorch_lightning\loops\utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
trainer.fit(model, train_dataloaders=train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 1.6 M 
-------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.427     Total estimated model params size (MB)


Loaded samples into dataset with label 'no'=0 and 'yes'=1
                                                                           

C:\Users\Julia\miniconda3\envs\ml4h\lib\site-packages\pytorch_lightning\trainer\trainer.py:1938: PossibleUserWarning: The number of training samples (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 17: 100%|██████████| 8/8 [01:20<00:00, 10.03s/it, loss=0.475, v_num=3]

C:\Users\Julia\miniconda3\envs\ml4h\lib\site-packages\pytorch_lightning\trainer\trainer.py:727: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
